In [1]:
import numpy as np
from tqdm import tqdm
import time
import math
import gc
import torch
from datasets import load_dataset
from transformers import GPTNeoXForCausalLM, GPTNeoXTokenizerFast, DataCollatorForLanguageModeling

/usr/local/Caskroom/miniconda/base/envs/reversing-llms/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from importlib import reload
import stationary_reversal as sr
reload(sr)

<module 'stationary_reversal' from '/Users/alexinf/Dropbox/github/reverse-dynamics-nlp/reverse-llm-benchmarking/stationary_reversal.py'>

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GPTNeoXForCausalLM.from_pretrained(
          "EleutherAI/pythia-160m-deduped-v0"
      ).to(device)
tokenizer = GPTNeoXTokenizerFast.from_pretrained("EleutherAI/gpt-neox-20b")

prefix_length  = 1
suffix = " Obama"
tokenized_suffix= tokenizer.encode(suffix, return_tensors="pt").to(device)
# tokenized_suffix = tokenized_suffix.unsqueeze(0)
suffix_length = len(tokenized_suffix[0])
empirical_dist = torch.load("../data/pile10k_empirical.pt")
vocab_size = empirical_dist.shape[0]

In [4]:
Obama_log_probs_batch = sr.stationary_reverse_full_dist(
  model, empirical_dist, prefix_length, tokenized_suffix)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable 

i= 0


TOKENIZERS_PARALLELISM=(true | false)
  0%|          | 0/32 [00:00<?, ?it/s]

100%|██████████| 32/32 [02:56<00:00,  5.50s/it]
/Users/alexinf/Dropbox/github/reverse-dynamics-nlp/reverse-llm-benchmarking/stationary_reversal.py:114: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  p = torch.distributions.Categorical(torch.nn.functional.softmax(vector_of_logprobs[prefix_length - i - 1,:])).sample()


In [13]:
# top k tokens for Obama_log_probs_batch
k = 10
top_k_tokens = torch.topk(Obama_log_probs_batch, k, dim=1).indices
top_k_tokens = top_k_tokens.cpu().numpy()
top_k_tokens = tokenizer.batch_decode(top_k_tokens, skip_special_tokens=True)
print(top_k_tokens)

[' Barack of the, and President in. that to']


In [17]:
reverse_model = GPTNeoXForCausalLM.from_pretrained(
  "afterless/reverse-pythia-160m"
).to(device)
reverse_model.eval()
rev_out = reverse_model(input_ids=tokenized_suffix)
rev_logits = rev_out.logits
rev_logprobs = torch.nn.functional.log_softmax(rev_logits, dim=-1)
rev_probs = torch.exp(rev_logprobs)

In [29]:
# top k tokens for rev_probs
k = 10
rev_topk = torch.topk(rev_probs, k, dim=-1)
rev_topk_tokens = rev_topk.indices
rev_topk_probs = rev_topk.values
rev_topk_words = tokenizer.decode(rev_topk_tokens[0,0,:])

In [30]:
rev_topk_words

' Barack of the and for that, an from by'

In [132]:
Obama_probs = torch.exp(Obama_log_probs_batch)
stat_total_variation = 0.5*torch.sum(torch.abs(Obama_probs[0,:] - rev_probs[0,0,:]))

In [39]:
uniform_dist = (1.0/vocab_size)+torch.zeros(vocab_size)
unif_total_variation = 0.5*torch.sum(torch.abs(uniform_dist - rev_probs[0,0,:]))

In [10]:
from importlib import reload
import stationary_reversal as sr
reload(sr)

tokenized_suffix_test = torch.cat((torch.zeros([1,20], dtype=torch.int32) ,tokenized_suffix),dim=-1)
test_suffix = sr.stationary_reverse_full_dist_suffix_calculation(
      model,
      empirical_dist, 
      tokenized_suffix_test,
      vocab_batch_size=786)

4024320
i= 0
3823104


  0%|          | 0/64 [00:00<?, ?it/s]

132048
3321271296
3144


  2%|▏         | 1/64 [00:06<06:27,  6.16s/it]

132048


  2%|▏         | 1/64 [00:11<11:52, 11.31s/it]


KeyboardInterrupt: 

In [7]:
tokenized_suffix_test.shape

torch.Size([1, 21])

In [133]:
0.5*(torch.sum(torch.abs(torch.exp(test_suffix[1,:])-Obama_probs)))

tensor(0.)

In [131]:
test_suffix.shape

torch.Size([2, 50304])